# Amazon SageMaker を使った学習済 Keras モデルのデプロイ
このノートブックは、[SageMaker TensorFlow Serving コンテナ](https://github.com/aws/sagemaker-tensorflow-serving-container) でリアルタイム推論を実行する方法を示します。 TensorFlow Servingコンテナは、スクリプトモードのデフォルトの推論方法です。より詳細なドキュメントについては、[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst)にアクセスしてください。

Amazon SageMaker python SDK でモデルをデプロイするためには、Estimator クラスの deploy（）メソッドを使って SageMaker モデルを作成します。このモデルはエンドポイントにデプロイされ、リアルタイムで予測リクエストを処理します。 スクリプトモードでトレーニングしたため、エンドポイントには TensorFlow Serving コンテナを使用します。 このサービングコンテナは、SageMaker ホスティングプロトコルと互換性のあるWebサーバーの実装を実行します。 独自の推論コードの使用 ドキュメントでは、SageMaker が推論コンテナを実行する方法について説明しています。

Keras/TensorFlow モデルをデプロイするためには、 TensorFlow SavedModel 形式でモデルを保存する必要があります。
本ハンズオンでは学習スクリプトの中で既に`def save_model(model, output):`として実装されています。


入力データと出力データの形式は、[TensorFlow Serving REST API](https://www.tensorflow.org/tfx/serving/api_rest) の Predictメソッドのリクエストとレスポンスの形式に直接対応しています。 SageMaker の TensforFlow Serving エンドポイントは、単純化されたJSON形式、行区切りのJSONオブジェクト ("jsons" または "jsonlines")、CSV データなど、TensorFlow REST API の一部ではない追加の入力形式も受け入れることができます。

## 学習済モデルのデプロイ
新しくモデルを学習する変わりに、これまでに学習されたモデルを活用しましょう。

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_sample1_CloudWatch.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge')


estimator = estimator.attach(training_job_name='cifar10-2019-09-30-13-35-29-611')  ## Configure with your previous cifar10 job name

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

## 推論の実行
エンドポイントが想定通りに動作するか、ランダムデータを用いて推論してみましょう。

In [ ]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)
print("Predicted class is {}".format(np.argmax(predictor.predict(data)['predictions'])))

### テストデートを用いたモデル精度の測定

テストデータセットを用いて混合行列を作成し、モデルの精度を測ってみましょう。

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
datagen = ImageDataGenerator()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def predict(data):
    predictions = predictor.predict(data)['predictions']
    return predictions

In [ ]:
batch_size = 128
predicted = []
actual = []
batches = 0
for data in datagen.flow(x_test,y_test,batch_size=batch_size):
    for i,prediction in enumerate(predict(data[0])):
        predicted.append(np.argmax(prediction))
        actual.append(data[1][i][0])
    batches += 1
    if batches >= len(x_test) / batch_size:
        break

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_pred=predicted,y_true=actual)
display('Average accuracy: {}%'.format(round(accuracy*100,2)))

In [ ]:
%matplotlib inline
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm = confusion_matrix(y_pred=predicted,y_true=actual)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sn.set(rc={'figure.figsize':(11.7,8.27)})
sn.set(font_scale=1.4)#for label size
sn.heatmap(cm, annot=True,annot_kws={"size": 10})# font size

ヒートマップを使うことで、それぞれのラベル毎の正解率を計算することが出来ます。

## エンドポイントを削除する
余分なコストが発生しないように、検証が終わったら上記で作成したエンドポイントを削除しましょう。

In [ ]:
sagemaker_session.delete_endpoint(predictor.endpoint)